# Stablepy

Install dependencies

In [ ]:
!pip install -q git+https://github.com/R3gm/stablepy.git@v0.2.0

Download our models and other stuffs

In [ ]:
# Model
!wget https://huggingface.co/frankjoshua/toonyou_beta6/resolve/main/toonyou_beta6.safetensors

# VAE
!wget https://huggingface.co/fp16-guy/anything_kl-f8-anime2_vae-ft-mse-840000-ema-pruned_blessed_clearvae_fp16_cleaned/resolve/main/anything_fp16.safetensors

# LoRAs
!wget https://civitai.com/api/download/models/183149 --content-disposition
!wget https://civitai.com/api/download/models/97655 --content-disposition

# Embeddings
!wget https://huggingface.co/embed/negative/resolve/main/bad-hands-5.pt
!wget https://huggingface.co/embed/negative/resolve/main/bad-artist.pt

# Upscaler
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth

## Inference with Stable diffusion 1.5

First, we pass the path of the model we will use.

The default task is txt2img but it can be changed to: openpose, canny, mlsd, scribble, softedge, segmentation, depth, normalbae, lineart, shuffle, ip2p or inpaint

In [ ]:
from stablepy import Model_Diffusers
import torch

model_path = "./toonyou_beta6.safetensors"
vae_path = "./anything_fp16.safetensors"

model = Model_Diffusers(
    base_model_id = model_path, # path to the model
    task_name = "canny", # task
    vae_model = vae_path, # path vae
)

To switch tasks, we can call `model.load_pipe()` and specify the new task or model. This will load the necessary components.

In [ ]:
model.load_pipe(
    base_model_id = model_path, # path to the model
    task_name = "txt2img", # task
    vae_model = None, # Use default VAE
)

We will use a basic txt2img task in which we can specify different common parameters, such as Loras, embeddings, upscaler, etc.

In [ ]:
from IPython.display import display

lora1_path = "./EmptyEyes_Diffuser_v10.safetensors"
lora2_path = "./FE_V2.safetensors" # pixel art lora
upscaler_path = "./RealESRGAN_x4plus.pth"

images, image_list = model(
    prompt = "pixel art (masterpiece, best quality), 1girl, collarbone, wavy hair, looking at viewer, blurry foreground, upper body, necklace, contemporary, plain pants, ((intricate, print, pattern)), ponytail, freckles, red hair, dappled sunlight, smile, happy,",
    negative_prompt = "(worst quality, low quality, letterboxed), bad_artist_token, bad_hand_token",
    img_width = 512,
    img_height = 1024,
    num_images = 1,
    num_steps = 30,
    guidance_scale = 8.0,
    clip_skip = True,
    seed = -1, # random seed
    sampler="DPM++ SDE Karras",
    syntax_weights="Compel",

    lora_A = lora1_path,
    lora_scale_A = 0.8,
    lora_B = lora2_path,
    lora_scale_B = 0.9,

    textual_inversion=[("bad_artist_token", "./bad-artist.pt"), ("bad_hand_token", "./bad-hands-5.pt")], # Is a list of tuples with [("<token_activation>","<path_embeding>"),...]

    upscaler_model_path = upscaler_path, # High res fix
    upscaler_increases_size=1.5,
)

for image in images:
  display(image)

# ControlNet

In [ ]:
model.load_pipe(
    base_model_id = model_path,
    task_name = "canny",
    # Use default VAE
)

Select a control image

In [ ]:
from PIL import Image

!wget https://huggingface.co/lllyasviel/sd-controlnet-canny/resolve/main/images/bird.png

control_image = "bird.png"
image = Image.open(control_image)
display(image)

Inference with canny

In [ ]:
images, image_list = model(
    prompt = "(masterpiece, best quality), bird",
    negative_prompt = "(worst quality, low quality, letterboxed)",
    image = control_image,
    # preprocessor_name="None", only canny not need the preprocessor_name, active by default
    preprocess_resolution = 512, # It is the resize of the image that will be obtained from the preprocessor.
    image_resolution = 768, # The equivalent resolution to be used for inference.
    controlnet_conditioning_scale = 1.0, # ControlNet Output Scaling in UNet
    control_guidance_start = 0.0, # ControlNet Start Threshold (%)
    control_guidance_end= 1.0, # ControlNet Stop Threshold (%)

    upscaler_model_path = upscaler_path,
    upscaler_increases_size=1.5,
)

for image in images:
  display(image)

valid `preprocessor_name` depending on the task: HED, HED safe, Midas, MLSD, Openpose, PidiNet, PidiNet safe, NormalBae, Lineart, Lineart coarse, LineartAnime, None (anime), ContentShuffle, DPT or UPerNet

# Adetailer

In [ ]:
model.load_pipe(
    base_model_id = model_path,
    task_name = "txt2img",
)

There must be a match of parameters for good results to be obtained with adetailer, it is also useful to use `strength` in adetailer_inpaint_params with low values ​​below 0.4.

In [ ]:
img_width = 512
img_height = 1024
num_steps = 30
CFG = 8.0

adetailer_inpaint_params = {
    "prompt": None, # main prompt will be use
    "negative_prompt" : None,
    "strength" : 0.35, # need low values
    "num_inference_steps": num_steps,
    "width": img_width,
    "height": img_height,
    "guidance_scale" : CFG,
}

adetailer_params = {
    "face_detector_ad" : True,
    "person_detector_ad" : True,
    "hand_detector_ad" : False,
    "inpaint_only" : adetailer_inpaint_params,
    "mask_dilation" : 4,
    "mask_blur" : 4,
    "mask_padding" : 32,
}

images, image_list = model(
    prompt = "(masterpiece, best quality), 1girl, collarbone, wavy hair, looking at viewer, blurry foreground, upper body, necklace, contemporary, plain pants, ((intricate, print, pattern)), ponytail, freckles, red hair, dappled sunlight, smile, happy,",
    negative_prompt = "(worst quality, low quality, letterboxed)",
    img_width = img_width,
    img_height = img_height,
    num_images = 1,
    num_steps = num_steps,
    guidance_scale = CFG,
    clip_skip = True,
    seed = 33,
    sampler="DPM++ SDE Karras",

    FreeU=True, # Improves diffusion model sample quality at no costs.
    adetailer_active=True,
    adetailer_params=adetailer_params,

    # upscaler_model_path = upscaler_path,
    # upscaler_increases_size=1.5,
)

for image in images:
  display(image)

In [ ]:
# For more details about the parameters
help(model.__call__)